In [16]:
import requests
import json
from bs4 import BeautifulSoup
from urllib.parse import urljoin, quote
from lxml import etree
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import datetime

In [5]:
jugador = "Lamine Yamal"
fecha_inicio = "2022-01-01"
fecha_fin = "2023-01-31"

url = f"https://news.google.com/search?q={jugador.replace(' ', '%20')}+after:{fecha_inicio}+before:{fecha_fin}"

response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

elements = soup.find_all('c-wiz', {'jsrenderer': 'ARwRbe'})

In [6]:
l_href = []

for element in elements:
    links = element.find_all('a')

    for link in links:
        href = link.get('href')

        if href:
            l_href.append(href)

print(l_href)


['https://support.google.com/googlenews?p=consumer_info&hl=en', './read/CBMivgFBVV95cUxOTE5rXzJNTl9PcEFXME41UzJwek1TMTFoeDllSlZsb2JuWjFvM3c1cUFCVDNlUF93TUt2cWNjNGlKWUloVm9acmdNbnAwWWxXZXkweFYzSkJoalNJdW9Ka09FUlR3TFlWVS1TM0hLRnFSRzIxOG1zcER4QV82RmxGR3djVFptMXJpZnpxcVYxQUQ4RkZEWXg0dWNGTExla2JCRURUeE92NVBybUNjZG9PZUhDd2hCMWZRMUlVbEJB0gHDAUFVX3lxTE5BM3NmRUVvR1pjTzJxeS16TkZrSjQ4cDJzcldMVUZFNTBLNnFfWnpKWENWVk9lRXAySVpaMHB5bXMzN3JxeFdVNmpMaDRGU25MQkVzbDA3eXhfQ1AydGR1cFk4RVlxazlzR1JvaXBoVmpUT0tNZXNnX2ZPUEJJS3BmdmxJTGhfdS1KS0UyNzFVRnZhY250eFo1dkROZ0NWb0RzN25OY1JyM0k0OGRwVkJuS1VrRkhSc1FnMGVFQzFwNGtLMA?hl=en-US&gl=US&ceid=US%3Aen', './read/CBMivgFBVV95cUxOTE5rXzJNTl9PcEFXME41UzJwek1TMTFoeDllSlZsb2JuWjFvM3c1cUFCVDNlUF93TUt2cWNjNGlKWUloVm9acmdNbnAwWWxXZXkweFYzSkJoalNJdW9Ka09FUlR3TFlWVS1TM0hLRnFSRzIxOG1zcER4QV82RmxGR3djVFptMXJpZnpxcVYxQUQ4RkZEWXg0dWNGTExla2JCRURUeE92NVBybUNjZG9PZUhDd2hCMWZRMUlVbEJB0gHDAUFVX3lxTE5BM3NmRUVvR1pjTzJxeS16TkZrSjQ4cDJzcldMVUZFNTBLNnFfWnpKWENWVk9lRXAySVpaMHB5bXMzN3JxeFdVNmp

In [30]:
def generate_urls(jugador, anio_inicio, mes_inicio, dia_inicio, anio_fin, mes_fin, dia_fin):
    # referencia https://news.google.com/search?q=nombre_jugador+de_futbol+after:YYYY-MM-DD+before:YYYY-MM-DD
    
    interval_tuples = []
    
    # se hacen intervalos de 1 semana
    while True:
        if anio_inicio == anio_fin and mes_inicio == mes_fin and dia_inicio >= dia_fin:
            break

        if anio_inicio == anio_fin and mes_inicio > mes_fin and dia_inicio:
            break

        if anio_inicio > anio_fin:
            break
        
        interval_tuples.append((anio_inicio, mes_inicio, dia_inicio))
        
        dia_inicio += 7
        
        if dia_inicio > 30:
            dia_inicio = 1
            mes_inicio += 1
        
        if mes_inicio > 12:
            mes_inicio = 1
            anio_inicio += 1

    print(interval_tuples)

    urls = []

    for interval in interval_tuples:
        anio, mes, dia = interval

        url = f"https://news.google.com/search?q={jugador.replace(' ', '%20')}+after:{anio}-{mes}-{dia}+before:{anio}-{mes}-{dia + 7}"
        urls.append(url)

    print(urls)

    return urls

root_urls = generate_urls("Lamine Yamal", 2022, 1, 1, 2023, 1, 31)

[(2022, 1, 1), (2022, 1, 8), (2022, 1, 15), (2022, 1, 22), (2022, 1, 29), (2022, 2, 1), (2022, 2, 8), (2022, 2, 15), (2022, 2, 22), (2022, 2, 29), (2022, 3, 1), (2022, 3, 8), (2022, 3, 15), (2022, 3, 22), (2022, 3, 29), (2022, 4, 1), (2022, 4, 8), (2022, 4, 15), (2022, 4, 22), (2022, 4, 29), (2022, 5, 1), (2022, 5, 8), (2022, 5, 15), (2022, 5, 22), (2022, 5, 29), (2022, 6, 1), (2022, 6, 8), (2022, 6, 15), (2022, 6, 22), (2022, 6, 29), (2022, 7, 1), (2022, 7, 8), (2022, 7, 15), (2022, 7, 22), (2022, 7, 29), (2022, 8, 1), (2022, 8, 8), (2022, 8, 15), (2022, 8, 22), (2022, 8, 29), (2022, 9, 1), (2022, 9, 8), (2022, 9, 15), (2022, 9, 22), (2022, 9, 29), (2022, 10, 1), (2022, 10, 8), (2022, 10, 15), (2022, 10, 22), (2022, 10, 29), (2022, 11, 1), (2022, 11, 8), (2022, 11, 15), (2022, 11, 22), (2022, 11, 29), (2022, 12, 1), (2022, 12, 8), (2022, 12, 15), (2022, 12, 22), (2022, 12, 29), (2023, 1, 1), (2023, 1, 8), (2023, 1, 15), (2023, 1, 22), (2023, 1, 29)]
['https://news.google.com/search?q=

In [32]:
def get_news_links(url):
    driver = webdriver.Edge()
    driver.get(url)

    # Espera a que el botón de 'Accept all' esté presente y clicable
    wait = WebDriverWait(driver, 10)  # Espera hasta 10 segundos
    accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Aceptar todo']")))

    # Hace clic en el elemento <span>
    accept_button.click()

    # Esperar a que la nueva página comience a cargarse o hacer algo en el medio si es necesario
    time.sleep(2)  # Pausa opcional para dar tiempo a la carga inicial de la página

    # Scroll hacia abajo hasta el final de la página
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
            # Desplazarse hacia abajo
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # Esperar a que se cargue la nueva sección
            time.sleep(2)  # Ajusta el tiempo según sea necesario para la página
            
            # Calcular la nueva altura de la página después del scroll
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            # Si la altura no cambia, hemos llegado al final de la página
            if new_height == last_height:
                break
            
            last_height = new_height

    elements = driver.find_elements(By.CSS_SELECTOR, 'c-wiz[jsrenderer="ARwRbe"]')

    # Extraer los href de cada elemento si tienen un enlace dentro
    links = []
    for element in elements:
        # Buscar el enlace dentro del elemento
        link = element.find_element(By.TAG_NAME, 'a')
        href = link.get_attribute('href')
        if href:
            links.append(href)





    # Encontrar todos los elementos <time> con class="hvbAAd"
    time_elements = driver.find_elements(By.CSS_SELECTOR, 'time.hvbAAd')

    # Extraer el valor del atributo 'datetime' de cada <time>
    datetimes = []
    for element in time_elements:
        datetime_value = element.get_attribute('datetime')
        if datetime_value:
            datetimes.append(datetime_value)







    driver.quit()

    return links[1:], datetimes


url = 'https://news.google.com/search?q=Lamine%20Yamal+after:2022-1-1+before:2022-1-8'

n_links, dates = get_news_links(url)
print(n_links)
print(dates)

['https://news.google.com/read/CBMigAJBVV95cUxQVm9zRGk3a0FNdTFHaVJoV3Q2bm9WV1FQNExCbE95QmdhSHNlSXU4ZkVzU0FuN3Y0YzhpX1N1dHFodUFBaUVMT21LVEFrR1g1ZlBfQlhIVzlabVJhdVZBdGozem12b04xOWpWa0c1b1NnT2pIU1h4LUcxWXFCWmt6YUI4LTI3c1lrZTVlMWd0cXRtbFdadXAyYzQtWlczalhTNzFpYWxIdDRwV1pXMHBPV21FaHc4Q1dOMUZWS212WGw3eWVjVUI1cGQ3bEExMGJWLW5HY2V3bkJldS1MbE9hbzNDTUFiVjRMZVVkcFhzeThrR1QxdTZKa216RUVFNXlS?hl=es&gl=ES&ceid=ES%3Aes', 'https://news.google.com/read/CBMitAFBVV95cUxOcHczMF9xbW5BWC10c3Zzdk8wclY4a2hveXJyaEVRZDNsR2dPUUxfbmNVczhYQmlwcjFwMUxVOVRqQ1lsLUlVRE8xLXBxMFotMTd2eTl2Y0h2RE9hTkZ4RTVabnl2cVlBckl6SDJKVkVKYW9PVmlnLTFJNTlZU195eFVoNzV4TXhpNFo1aEk5c3V2ckp6QUc5cFo1dEVSTDlGR2ptVHBpN0tQcmN0TlJRRndxWU7SAboBQVVfeXFMTWZEdGs5SGVBUGZBeFZxMGZSenF6SWtTUWFnZjJRWjBPNkNpRHdpOFlhZHR2VTZWRzdtNG1IR2lKN2xrNHVrdVJQWGpDM0NGeXZfdFhfZHhWVW9DaDR6ZkdTYlRZellpai1YeTFhRmR6ZVdFZXQxNmRkRzBUMGJnZGkwelBZX3YtMXVkM0t1ZDV2ZjdSYk9SVDJpT3RxT3RXcnNET291TDQ2Ujd2YTF1aE80UVRpWU9ueDdB?hl=es&gl=ES&ceid=ES%3Aes', 'https://news.google.com/read/CBMiv

In [ ]:
def get_news_parragraphs(url):
    driver = webdriver.Edge()

    driver.get(url)

    try:
        # Espera a que el botón de 'Accept all' esté presente y clicable
        wait = WebDriverWait(driver, 10)  # Espera hasta 10 segundos
        accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Accept all']")))
        
        # Hace clic en el elemento <span>
        accept_button.click()

        # Esperar a que la nueva página comience a cargarse o hacer algo en el medio si es necesario
        time.sleep(2)  # Pausa opcional para dar tiempo a la carga inicial de la página

        # Scroll hacia abajo hasta el final de la página
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # Desplazarse hacia abajo
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # Esperar a que se cargue la nueva sección
            time.sleep(2)  # Ajusta el tiempo según sea necesario para la página
            
            # Calcular la nueva altura de la página después del scroll
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            # Si la altura no cambia, hemos llegado al final de la página
            if new_height == last_height:
                break
            
            last_height = new_height
        
        # Una vez que se ha llegado al final, obtener todos los párrafos <p>
        paragraphs = driver.find_elements(By.TAG_NAME, 'p')
        
        # Imprimir el texto de cada párrafo
        for paragraph in paragraphs:
            return paragraph.text

        
    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.quit()
